## Simple Autoencoder

Simple tutorial to train your first [autoencoder](https://en.wikipedia.org/wiki/Autoencoder) in Torch.

As a toy dataset, we will use the common MNIST dataset. More information about MNIST can be found at http://yann.lecun.com/exdb/mnist/.

First, we import some dependecies.

In [1]:
require 'torch'
require 'nn'
require 'optim'
mnist = require 'mnist'

We load and shuffle the data.

In [2]:
train_data = mnist.traindataset()
test_data = mnist.testdataset()

perm = torch.randperm(train_data.label:size(1)):long()
train_data.data = train_data.data:index(1, perm):float()
train_data.label = train_data.label:index(1, perm)

test_data.data = test_data.data:float()

We print one image as an example.

In [3]:
itorch.image(train_data.data[613])

We create the network and print the network's architecture.

In [4]:
function create_model()
    net = nn.Sequential()
    
    -- encoder
    net:add(nn.Reshape(28*28)) -- flatten the image
    net:add(nn.Linear(28*28, 512)) -- first fully connected layer
    net:add(nn.ReLU()) -- non-linearity
    
    -- code / low-dimensional representation
    net:add(nn.Linear(512, 128)) -- second fully connected layer
    net:add(nn.ReLU())
    
    -- decoder
    net:add(nn.Linear(128, 512)) -- third fc layer
    net:add(nn.ReLU())
    net:add(nn.Linear(512, 28*28)) -- fourth fc layer
    
    net:add(nn.Reshape(28, 28))
    return net
end

In [5]:
model = create_model()
print(model:__tostring())

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> output]
  (1): nn.Reshape(784)
  (2): nn.Linear(784 -> 512)
  (3): nn.ReLU
  (4): nn.Linear(512 -> 128)
  (5): nn.ReLU
  (6): nn.Linear(128 -> 512)
  (7): nn.ReLU
  (8): nn.Linear(512 -> 784)
  (9): nn.Reshape(28x28)
}	


We create the loss function. We use the *mean squared error* (MSE) to measure the difference between the input and its reconstruction. We aim at minimizing the MSE.

In [6]:
criterion = nn.MSECriterion():float()

Defining now hyperparameters related to the network training.

In [7]:
optimState = {learningRate=0.01, momentum=0.5} -- parameters for the (minibatch) gradient descent algorithm

batchSize = 128
maxEpoch = 20 -- train for 20 epochs

display_n = 500000 -- display training loss and accuracy every x step
test_n = 500 -- test the model every x step

In [8]:
-- placeholder for batch
X_batch = train_data.data[{{1,batchSize}}]:clone():float()

-- placeholder for batch indices
batchIndices = torch.LongTensor(batchSize)

Optionally, we can cast the model created as well as the different variables on GPU.

In [9]:
-- put on GPU
require 'cutorch'
require 'cunn'
cutorch.setDevice(1) -- chose which GPU to use
model = model:cuda()
criterion = criterion:cuda()
train_data.data = train_data.data:cuda()
test_data.data = test_data.data:cuda()
X_batch = X_batch:cuda()

We now train the network.

In [10]:
-- model parameters for optimization
local params, gradParams = model:getParameters()
local iteration = 0

local steps_per_epoch = train_data.label:size(1) / batchSize

while true do
   iteration = iteration + 1
    
   batchIndices:random(1, train_data.label:size(1))
   X_batch:copy(train_data.data:index(1, batchIndices))

    ----------------------------------------
    -------------- TRAINING ----------------
    ----------------------------------------
   function feval(params)
      gradParams:zero()
      outputs = model:forward(X_batch)
      loss = criterion:forward(outputs, X_batch)
      local gradOutputs = criterion:backward(outputs, X_batch)
      model:backward(X_batch, gradOutputs)
      return loss, gradParams
   end
    
   timer = torch.Timer() -- timer
   optim.adagrad(feval, params, optimState)
   datum_sec = batchSize / 
   timer:time().real
    
   if iteration % display_n == 0 then
          print(string.format("TRAINING – epoch %.2f, loss = %.4f, %.2f datum/sec", iteration / steps_per_epoch, loss, datum_sec))
   end
    ----------------------------------------
    ----------------------------------------
    
    
    ----------------------------------------
    ------------- EVALUATION ---------------
    ----------------------------------------
   if (iteration - 1) % test_n == 0 or iteration / steps_per_epoch >= maxEpoch then
       local outputs_ = model:forward(test_data.data)
       local loss_ = criterion:forward(outputs, test_data.data)
       print(string.format('TEST – epoch %.2f, loss: %.4f', iteration / steps_per_epoch, loss_))
   end
    ----------------------------------------
    ----------------------------------------
    
    if iteration / steps_per_epoch > maxEpoch then
        break
   end
end

TEST – epoch 0.00, loss: 263610.7812	


TEST – epoch 1.07, loss: 1511.2645	


TEST – epoch 2.14, loss: 1194.8921	


TEST – epoch 3.20, loss: 1011.7727	


TEST – epoch 4.27, loss: 930.7047	


...

TEST – epoch 17.07, loss: 639.2548	


TEST – epoch 18.14, loss: 624.9268	


TEST – epoch 19.20, loss: 610.5811	


TEST – epoch 20.00, loss: 603.2516	


TEST – epoch 20.00, loss: 606.5347	


Let's see how our trained autoencoder reconstruct some images.

In [12]:
for i=1,10 do
    original = test_data.data[torch.random(test_data.data:size(1))]
    reconstruction = net:forward(original)
    itorch.image({original, reconstruction})
end

The reconstructions are not perfect. In the notebook [autoencoder with CNN](autoencoder-cnn.ipynb), we achieve better results when using convolutional neural networks instead of fully connected layers for the encoder and the decoder.